In [19]:
from ib_insync import *
util.startLoop()  # uncomment this line when in a notebook

ib = IB()
ib.connect('127.0.0.1', 7497, clientId=101)

<IB connected to 127.0.0.1:7497 clientId=101>

# market order

In [20]:
m=MarketOrder('BUY',1)
c=ib.qualifyContracts(Stock('SPY','SMART','USD'))[0]
c

Stock(conId=756733, symbol='SPY', exchange='SMART', primaryExchange='ARCA', currency='USD', localSymbol='SPY', tradingClass='SPY')

In [21]:
po=ib.placeOrder(c,m)
print(po)

Trade(contract=Stock(conId=756733, symbol='SPY', exchange='SMART', primaryExchange='ARCA', currency='USD', localSymbol='SPY', tradingClass='SPY'), order=MarketOrder(orderId=4, clientId=101, action='BUY', totalQuantity=1), orderStatus=OrderStatus(orderId=4, status='PendingSubmit', filled=0.0, remaining=0.0, avgFillPrice=0.0, permId=0, parentId=0, lastFillPrice=0.0, clientId=0, whyHeld='', mktCapPrice=0.0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2024, 1, 31, 12, 58, 12, 766545, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='', errorCode=0)], advancedError='')


# limit order

In [22]:
m=LimitOrder('BUY',1,1600)
c=ib.qualifyContracts(Stock('KOTAKBANK','NSE'))[0]
print(m,c)

LimitOrder(action='BUY', totalQuantity=1, lmtPrice=1600) Stock(conId=78743661, symbol='KOTAKBANK', exchange='NSE', primaryExchange='NSE', currency='INR', localSymbol='KOTAKBANK', tradingClass='KOTAKBANK')


In [23]:
po1=ib.placeOrder(c,m)

In [24]:
po1

Trade(contract=Stock(conId=78743661, symbol='KOTAKBANK', exchange='NSE', primaryExchange='NSE', currency='INR', localSymbol='KOTAKBANK', tradingClass='KOTAKBANK'), order=LimitOrder(orderId=6, clientId=101, action='BUY', totalQuantity=1, lmtPrice=1600), orderStatus=OrderStatus(orderId=6, status='PendingSubmit', filled=0.0, remaining=0.0, avgFillPrice=0.0, permId=0, parentId=0, lastFillPrice=0.0, clientId=0, whyHeld='', mktCapPrice=0.0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2024, 1, 31, 12, 58, 13, 502766, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='', errorCode=0)], advancedError='')

# stop order

In [25]:
c=ib.qualifyContracts(Stock('ONGC','NSE'))[0]
sp=StopOrder('BUY',1,250)
print(c,sp)


Stock(conId=83845926, symbol='ONGC', exchange='NSE', primaryExchange='NSE', currency='INR', localSymbol='ONGC', tradingClass='ONGC') StopOrder(action='BUY', totalQuantity=1, auxPrice=250)


In [26]:
ib.placeOrder(c,sp)

Trade(contract=Stock(conId=83845926, symbol='ONGC', exchange='NSE', primaryExchange='NSE', currency='INR', localSymbol='ONGC', tradingClass='ONGC'), order=StopOrder(orderId=8, clientId=101, action='BUY', totalQuantity=1, auxPrice=250), orderStatus=OrderStatus(orderId=8, status='PendingSubmit', filled=0.0, remaining=0.0, avgFillPrice=0.0, permId=0, parentId=0, lastFillPrice=0.0, clientId=0, whyHeld='', mktCapPrice=0.0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2024, 1, 31, 12, 58, 14, 240780, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='', errorCode=0)], advancedError='')

# cover order

In [27]:
price=3000
c=ib.qualifyContracts(Stock('ADANIENT','NSE'))[0]
print(c)
parent = Order(orderId=ib.client.getReqId(),action='BUY',orderType='LMT',totalQuantity=1,lmtPrice=price,transmit = False)


stopLoss = Order()
stopLoss.orderId = parent.orderId + 2
stopLoss.action = "SELL" if parent.action == "BUY" else "BUY"
stopLoss.orderType = "STP"
stopLoss.auxPrice = price-10
stopLoss.totalQuantity = parent.totalQuantity
stopLoss.parentId = parent.orderId
stopLoss.transmit = True

ords=[parent, stopLoss]

for o in ords:
    trade=ib.placeOrder(c, o)
    print(trade)

Stock(conId=56986798, symbol='ADANIENT', exchange='NSE', primaryExchange='NSE', currency='INR', localSymbol='ADANIENT', tradingClass='ADANIENT')
Trade(contract=Stock(conId=56986798, symbol='ADANIENT', exchange='NSE', primaryExchange='NSE', currency='INR', localSymbol='ADANIENT', tradingClass='ADANIENT'), order=Order(orderId=10, clientId=101, action='BUY', totalQuantity=1, orderType='LMT', lmtPrice=3000, transmit=False), orderStatus=OrderStatus(orderId=10, status='PendingSubmit', filled=0.0, remaining=0.0, avgFillPrice=0.0, permId=0, parentId=0, lastFillPrice=0.0, clientId=0, whyHeld='', mktCapPrice=0.0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2024, 1, 31, 12, 58, 14, 943087, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='', errorCode=0)], advancedError='')
Trade(contract=Stock(conId=56986798, symbol='ADANIENT', exchange='NSE', primaryExchange='NSE', currency='INR', localSymbol='ADANIENT', tradingClass='ADANIENT'), order=Order(orderId=12, clientId=101, acti

# bracket order

In [28]:
price=2890
c=ib.qualifyContracts(Stock('RELIANCE','NSE','INR'))[0]
print(c)

parent = Order(
    orderId=ib.client.getReqId(),
    action='BUY',
    orderType='LMT',
    totalQuantity=1,
    lmtPrice=price,
    transmit = False)


takeProfit = Order()        
takeProfit.orderId = ib.client.getReqId()
takeProfit.action = "SELL" if parent.action == "BUY" else "BUY"
takeProfit.orderType = "LMT"
takeProfit.totalQuantity = parent.totalQuantity
takeProfit.lmtPrice = price+3
takeProfit.parentId = parent.orderId
takeProfit.transmit = False


stopLoss = Order()
stopLoss.orderId = ib.client.getReqId()
stopLoss.action = "SELL" if parent.action == "BUY" else "BUY"
stopLoss.orderType = "STP"
stopLoss.auxPrice = price-3
stopLoss.totalQuantity = parent.totalQuantity
stopLoss.parentId = parent.orderId
stopLoss.transmit = True

ords=[parent, takeProfit, stopLoss]

for o in ords:
    trade=ib.placeOrder(c, o)
    print(trade)


Stock(conId=44652000, symbol='RELIANCE', exchange='NSE', primaryExchange='NSE', currency='INR', localSymbol='RELIANCE', tradingClass='RELIANCE')
Trade(contract=Stock(conId=44652000, symbol='RELIANCE', exchange='NSE', primaryExchange='NSE', currency='INR', localSymbol='RELIANCE', tradingClass='RELIANCE'), order=Order(orderId=13, clientId=101, action='BUY', totalQuantity=1, orderType='LMT', lmtPrice=2890, transmit=False), orderStatus=OrderStatus(orderId=13, status='PendingSubmit', filled=0.0, remaining=0.0, avgFillPrice=0.0, permId=0, parentId=0, lastFillPrice=0.0, clientId=0, whyHeld='', mktCapPrice=0.0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2024, 1, 31, 12, 58, 15, 743394, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='', errorCode=0)], advancedError='')
Trade(contract=Stock(conId=44652000, symbol='RELIANCE', exchange='NSE', primaryExchange='NSE', currency='INR', localSymbol='RELIANCE', tradingClass='RELIANCE'), order=Order(orderId=14, clientId=101, acti

# trailing order

In [29]:
price=1400
c=ib.qualifyContracts(Stock('HDFCBANK','NSE'))[0]
print(c)

order = Order()
order.orderId = ib.client.getReqId()
order.action = 'SELL'
order.orderType = "TRAIL"
order.totalQuantity = 1
order.trailingPercent = 2
order.trailStopPrice = int(price*0.98)
trade=ib.placeOrder(c, order)
print(trade)

Stock(conId=382750910, symbol='HDFCBANK', exchange='NSE', primaryExchange='NSE', currency='INR', localSymbol='HDFCBANK', tradingClass='HDFCBANK')
Trade(contract=Stock(conId=382750910, symbol='HDFCBANK', exchange='NSE', primaryExchange='NSE', currency='INR', localSymbol='HDFCBANK', tradingClass='HDFCBANK'), order=Order(orderId=17, clientId=101, action='SELL', totalQuantity=1, orderType='TRAIL', trailStopPrice=1372, trailingPercent=2), orderStatus=OrderStatus(orderId=17, status='PendingSubmit', filled=0.0, remaining=0.0, avgFillPrice=0.0, permId=0, parentId=0, lastFillPrice=0.0, clientId=0, whyHeld='', mktCapPrice=0.0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2024, 1, 31, 12, 58, 16, 650420, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='', errorCode=0)], advancedError='')


In [30]:
ib.client.getReqId()

18

# cancel order

In [31]:
def close_all_orders():
    orders = ib.reqOpenOrders()
    df=util.df(orders)

    df.drop('contract',axis=1,inplace=True)
    for ord in df.order.to_list():
        print(ord)
        b=ib.cancelOrder(ord)
        print(b)
    return 1

In [ ]:
def close_all_orders():
    orders = ib.reqAllOpenOrders()
    print(orders)
    for ord in orders:
        b=ib.cancelOrder(ord.order)
        print(b)
    return 1

In [ ]:
close_all_orders()

Order(orderId=14, clientId=101, permId=1143325781, action='SELL', totalQuantity=1.0, orderType='LMT', lmtPrice=2893.0, auxPrice=0.0, transmit=False, parentId=13)
Trade(contract=Stock(conId=44652000, symbol='RELIANCE', exchange='NSE', primaryExchange='NSE', currency='INR', localSymbol='RELIANCE', tradingClass='RELIANCE'), order=Order(orderId=14, clientId=101, permId=1143325781, action='SELL', totalQuantity=1.0, orderType='LMT', lmtPrice=2893.0, auxPrice=0.0, transmit=False, parentId=13), orderStatus=OrderStatus(orderId=14, status='PendingCancel', filled=0.0, remaining=1.0, avgFillPrice=0.0, permId=1143325781, parentId=13, lastFillPrice=0.0, clientId=101, whyHeld='child', mktCapPrice=0.0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2024, 1, 31, 12, 58, 15, 743394, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='', errorCode=0), TradeLogEntry(time=datetime.datetime(2024, 1, 31, 12, 58, 16, 316310, tzinfo=datetime.timezone.utc), status='PreSubmitted', message='', e

1

Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1102, reqId -1: Connectivity between IB and Trader Workstation has been restored - data maintained. All data farms are connected: uscrypto; usfarm.nj; hfarm; usfuture; cashfarm; usopt; usfarm; euhmds; hkhmds; ushmds; fundfarm; secdefhk.
Error 1102, reqId -1: Connectivity between IB and Trader Workstation has been restored - data maintained. All data farms are connected: uscrypto; usfarm.nj; hfarm; usfuture; cashfarm; usopt; usfarm; euhmds; hkhmds; ushmds; fundfarm; secdefhk.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100

In [ ]:
util.df(ib.positons())